In [10]:
import torch
import pandas as pd
import os
from torchvision import transforms
from EmotionDataset import EmotionDataset, get_transform
from EmotionCNN import EmotionCNN
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import matplotlib as plt

In [11]:
# Load and prepare the dataset
dataset_path = os.path.join(os.getcwd(), 'data')
labels_path = os.path.join(dataset_path, 'labels.csv')
df = pd.read_csv(labels_path)

df['label'] = df['pth'].apply(lambda x: x.split('/')[0])
df['pth'] = df['pth'].apply(lambda x: os.path.join(dataset_path, x))
one_hot_encoded_df = pd.get_dummies(df['label']).astype(int)
df = pd.concat([df.drop('label', axis=1), one_hot_encoded_df], axis=1)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.drop(columns=['relFCs'], inplace=True)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [12]:
# Create the datasets
transform = get_transform()
train_dataset = EmotionDataset(train_df, transform)
test_dataset = EmotionDataset(test_df, transform)

In [13]:
# Set up the model and training parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionCNN().to(device)
criterion = nn.BCEWithLogitsLoss()  # Changed loss function for one-hot encoded labels
optimizer = Adam(model.parameters(), lr=0.001)

In [14]:
# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [15]:
def calculate_accuracy(outputs, labels):
    _, predictions = torch.max(outputs, 1)
    _, labels = torch.max(labels, 1)
    return (predictions == labels).float().mean().item()

In [17]:
# Lists to store metrics
train_losses = []
train_accuracies = []
test_accuracies = []

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    # Save training loss
    train_losses.append(epoch_loss / len(train_loader))

    # Evaluate on training set
    model.eval()
    train_accuracy = 0
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            train_accuracy += calculate_accuracy(outputs, labels)
    train_accuracy /= len(train_loader)
    train_accuracies.append(train_accuracy)
    
    # Evaluate on testing set
    test_accuracy = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            test_accuracy += calculate_accuracy(outputs, labels)
    test_accuracy /= len(test_loader)
    test_accuracies.append(test_accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_losses[-1]:.4f}, '
          f'Training Accuracy: {train_accuracies[-1]:.4f}, Testing Accuracy: {test_accuracies[-1]:.4f}')

Epoch [1/10], Loss: 0.2497, Training Accuracy: 0.5659, Testing Accuracy: 0.5383
Epoch [2/10], Loss: 0.2155, Training Accuracy: 0.6434, Testing Accuracy: 0.5880


In [ ]:
# Plot model evaluation
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(epochs, test_accuracies, label='Testing Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Save model
torch.save(model, './models/emotionModel2')

In [ ]:
# Load model
model_path = './models/emotionModel1'
model = torch.load(model_path)